In [1]:
import nltk
from nltk.stem.porter import *
from torch.nn import *
from torch.optim import *
import numpy as np
import pandas as pd
import torch,torchvision
import random
from tqdm import *
from torch.utils.data import Dataset,DataLoader
stemmer = PorterStemmer()

In [2]:
def tokenize(sentence):
    return nltk.word_tokenize(sentence)

In [3]:
tokenize('$1000')

['$', '1000']

In [4]:
def stem(word):
    return stemmer.stem(word.lower())

In [5]:
stem('universe')

'univers'

In [6]:
def bag_of_words(tokenized_words,all_words):
    tokenized_words = [stem(w) for w in tokenized_words]
    bag = np.zeros(len(all_words),dtype=np.float32)
    for idx,w in enumerate(all_words):
        if w in tokenized_words:
            bag[idx] = 1.0
    return bag

In [7]:
bag_of_words(['how','arte','ate'],['ate','rgger','fewger'])

array([1., 0., 0.], dtype=float32)

In [8]:
data = pd.read_csv('./train_data_cleaning.csv')

In [9]:
X = data['text']

In [10]:
y = data['target']

In [11]:
all_words = []

In [12]:
all_data = []

In [13]:
tags = []

In [14]:
for X_batch,y_batch in tqdm(zip(X,y)):
    X_batch = tokenize(X_batch)
    new_X = []
    for X_batch_in_X_batch in X_batch:
        new_X.append(stem(X_batch_in_X_batch))
    all_words.extend(new_X)
    all_data.append((new_X,y_batch))
    tags.append(y_batch)

7613it [00:02, 3330.41it/s]


In [15]:
np.random.shuffle(all_data)

In [16]:
all_words = sorted(set(all_words))

In [17]:
tags = sorted(set(tags))

In [18]:
X = []
y = []

In [19]:
for sentence,tag in tqdm(all_data):
    bag = bag_of_words(sentence,all_words)
    X.append(bag)
    y.append(tags.index(tag))

100%|██████████████████████████████████████| 7613/7613 [00:16<00:00, 452.84it/s]


In [20]:
X[0]

array([0., 0., 0., ..., 0., 0., 0.], dtype=float32)

In [21]:
y[0]

0

In [22]:
from sklearn.model_selection import *

In [23]:
X_train,X_test,y_train,y_test = train_test_split(X,y,test_size=0.25,shuffle=False)

In [24]:
device = 'cuda'

In [25]:
X_train = torch.from_numpy(np.array(X_train)).to(device).float()
y_train = torch.from_numpy(np.array(y_train)).to(device).to(device).float()
X_test = torch.from_numpy(np.array(X_test)).to(device).to(device).float()
y_test = torch.from_numpy(np.array(y_test)).to(device).to(device).float()

In [26]:
def get_loss(model,X,y,criterion):
    preds = model(X)
    loss = criterion(preds.view(-1,1),y.view(-1,1))
    return loss.item()

In [27]:
def get_accuracy(model,X,y):
    correct = 0
    total = 0
    preds = model(X)
    for pred,y_batch in zip(preds,y):
        pred = int(torch.round(pred))
        if pred == y_batch:
            correct += 1
        total += 1
    acc = round(correct/total,3)*100
    return acc

In [36]:
class Model(Module):
    def __init__(self):
        super().__init__()
        self.activation = ReLU()
        self.linear1 = Linear(len(all_words),1024)
        self.linear2 = Linear(1024,1024)
        self.linear3 = Linear(1024,1)
    
    def forward(self,X):
        preds = self.linear1(X)
        for _ in range(10):
            preds = self.activation(self.linear2(preds))
        preds = self.linear3(preds)
        return preds

In [37]:
model = Model().to(device)

In [38]:
criterion = MSELoss()

In [39]:
optimizer = Adam(model.parameters(),lr=0.001)

In [40]:
epochs = 1000

In [41]:
batch_size = 32

In [42]:
import wandb

In [ ]:
wandb.init(project='NLP-with-Disaster-Tweets-Clf-V6',name='baseline')
wandb.watch(model)
for _ in tqdm(range(epochs)):
    for idx in range(0,len(X_train),batch_size):
        X_batch = X_train[idx:idx+batch_size].to(device)
        y_batch = y_train[idx:idx+batch_size].to(device)
        preds = model(X_batch)
        preds = preds.to(device)
        loss = criterion(preds.view(-1,1),y_batch.view(-1,1))
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
    wandb.log({'Loss':get_loss(model,X_train,y_train,criterion)})
    wandb.log({'Val Loss':get_loss(model,X_test,y_test,criterion)})
    wandb.log({'Acc':get_accuracy(model,X_train,y_train)})
    wandb.log({'Val Acc':get_accuracy(model,X_test,y_test)})
wandb.watch(model)
wandb.finish()

 24%|█████████▍                              | 235/1000 [06:34<21:16,  1.67s/it]